# Visualize a RadarKit netCDF Product

In [ ]:
import os
import datetime
import numpy as np

from netCDF4 import Dataset

import chart

In [ ]:
# file = os.path.expanduser('K:/OU/horus-open/data/PX-20230130-173637-E4.0-Z.nc')
# file = os.path.expanduser('K:/OU/horus-open/data/RAXPOL-20210930-174106-A45.0-Z.nc')
file = os.path.expanduser('~/Downloads/data/moment/20210930/RAXPOL-20210930-174105-A45.0-Z.nc')

## Load netCDF

In [ ]:
with Dataset(file, mode='r') as nc:
    name = nc.getncattr('TypeName')
    elev = np.array(nc.variables['Elevation'][:], dtype=np.float32)
    azim = np.array(nc.variables['Azimuth'][:], dtype=np.float32)
    gatewidth = np.array(nc.variables['GateWidth'][:], dtype=np.float32)
    values = np.array(nc.variables[name][:], dtype=np.float32)
    values[values < -90] = np.nan
    longitude = nc.getncattr('Longitude')
    latitude = nc.getncattr('Latitude')
    height = nc.getncattr('Height')
    sweepElev = nc.getncattr('Elevation')
    sweepAzim = nc.getncattr('Azimuth')
    sweepTime = nc.getncattr('Time')
    scantype = nc.getncattr('ScanType')
    symbol = file.split('.')[-2].split('-')[-1]

## Overlay for the Plot

In [ ]:
overlay = chart.atlas.Overlay((longitude, latitude), scantype=scantype)

In [ ]:
radii = np.concatenate(([1.0], np.arange(10.0, 31.0, 10.0)))
# radii = np.arange(0.0, 31.0, 10.0)
overlay.setRingRadii(radii)
overlay.load()

In [ ]:
# r = 1.0e-3 * np.arange(radar.values.shape[1]) * radar.gatewidth[0]
r = 1.0e-3 * np.arange(values.shape[1]) * gatewidth[0]
a = np.deg2rad(azim)
e = np.deg2rad(elev)
t = datetime.datetime.utcfromtimestamp(sweepTime)
timestr = t.strftime('%Y/%m/%d %H:%M:%S')
title = f'{timestr} UTC  ' + (f'EL: {sweepElev:.2f}°' if scantype == 'PPI' else f'AZ: {sweepAzim:.2f}°')
symbol = 'Z'

## Chart with Overlay

In [ ]:
ppi = chart.chart.Image(e, a, r, values, style=symbol, title=title, overlay=overlay, figsize=(800, 600), maxrange=30.0, scantype=scantype)
ppi.image()

## Convenient Function to Populate Image

In [ ]:
def populateImage(file):
    with Dataset(file, mode='r') as nc:
        name = nc.getncattr('TypeName')
        values = np.array(nc.variables[name][:], dtype=np.float32)
        values[values < -90] = np.nan
        symbol = file.split('.')[-2].split('-')[-1]
    ppi.set_data(values, style=symbol)

# def populateImage(file):
#     with Dataset(file, mode='r') as nc:
#         name = nc.getncattr('TypeName')
#         values = np.array(nc.variables[name][:], dtype=np.float32)
#         values[values < -90] = np.nan
#         symbol = file.split('.')[-2].split('-')[-1]
#         gatewidth = np.array(nc.variables['GateWidth'][:], dtype=np.float32)
#         azim = np.array(nc.variables['Azimuth'][:], dtype=np.float32)
#         r = 1.0e-3 * np.arange(values.shape[1]) * gatewidth[0]
#         a = np.deg2rad(azim)
#     ppi.set_data(values, style=symbol, a=a, r=r)

# populateImage(os.path.expanduser('~/Downloads/data/moment/20230130/PX-20230130-173637-E4.0-Z.nc'))
# populateImage(os.path.expanduser('~/Downloads/data/moment-ppc/20230130/PX-20230130-173637-E4.0-Z.nc'))
# populateImage(os.path.expanduser('~/Downloads/data/moment/20221208/RAXPOL-20221208-002501-E2.3-V.nc'))

## More Images

In [ ]:
# populateImage(os.path.expanduser('~/Downloads/data/moment/20230130/PX-20230130-173637-E4.0-Z.nc'))
# ppi.image()

In [ ]:
# populateImage(os.path.expanduser('~/Downloads/data/moment-nt/20230605/PX-20230605-115953-E4.0-Z.nc')); ppi.image()